In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from numpy import asarray
import glob
import cv2
import tensorflow as tf
from tensorflow import keras

## Import Data

### Train.csv

In [2]:
all_labels=pd.read_csv('train.csv')

In [3]:
Labels_DF=pd.DataFrame(all_labels)
Labels_DF.rename(columns={'image':'file_name'}, inplace=True)
Labels_DF.head(3)

,file_name,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b


In [4]:
#Labels_DF['species'].value_counts()

In [4]:
## species with incorrect spellings.  bottlenose_dolpin, kiler_whale.
Labels_DF['species'].replace('bottlenose_dolpin', 'bottlenose_dolphin', inplace=True)
Labels_DF['species'].replace('kiler_whale', 'killer_whale', inplace=True)

In [5]:
## create subcategory for whale or dolphin
sub=[]
for each in Labels_DF['species']:
    if 'dolphin' in each:
        a=1
    else: a=2
    sub.append(a)

In [6]:
Labels_DF['sub_cat']=sub

In [7]:
## create species cat numerical
from sklearn.preprocessing import OrdinalEncoder
OE=OrdinalEncoder()
SE=OE.fit_transform(Labels_DF[['species']])
Labels_DF['species_category']=SE.astype(int)

In [8]:
Labels_DF.head(6)

,file_name,species,individual_id,sub_cat,species_category
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,2,16
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,2,13
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,2,8
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,1,2
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,2,13
5,000a8f2d5c316a.jpg,bottlenose_dolphin,b9907151f66e,1,2


In [9]:
len(Labels_DF['species_category'].value_counts())

28

In [42]:
Labels_DF['species_category'].value_counts()

2     10781
0      7443
13     7392
1      4830
8      3326
7      3139
14     2455
25     1700
16     1689
17     1608
9      1324
12     1123
24      866
26      490
22      428
23      367
5       347
6       341
19      262
15      238
27      229
3       154
18      145
11      116
4        90
20       76
21       60
10       14
Name: species_category, dtype: int64

In [25]:
Species_Key=pd.DataFrame({
    'Species': Labels_DF['species'].unique(),
    'Key': Labels_DF['species_category'].unique()
})

In [26]:
Species_Key

,Species,Key
0,melon_headed_whale,16
1,humpback_whale,13
2,false_killer_whale,8
3,bottlenose_dolphin,2
4,beluga,0
5,minke_whale,17
6,fin_whale,9
7,blue_whale,1
8,gray_whale,12
9,southern_right_whale,24


### Separate images into Dolphins and Whales folders

In [10]:
Dolphins=pd.DataFrame(Labels_DF.loc[Labels_DF['sub_cat']==1])
Dolphins=Dolphins['file_name']
len(Dolphins)

16995

In [11]:
# for each in Dolphins:
#     try:
#         name=each
#         img=Image.open(f"C:\\Users\\chris\\OneDrive\\Documents\\Extracurricular_projects\\Kaggle_Marine_Life\\train_images/{name}")
#         img.save(f"Labels/Dolphins_Full/{name}.jpg")
#     except:
#         print(f"No file found {name}")

In [13]:
Whales=pd.DataFrame(Labels_DF.loc[Labels_DF['sub_cat']==2])
Whales=Whales['file_name']
len(Whales)

34038

In [45]:
# for each in Whales:
#     try:
#         name=each
#         img=Image.open(f"C:\\Users\\chris\\OneDrive\\Documents\\Extracurricular_projects\\Kaggle_Marine_Life\\train_images/{name}")
#         img.save(f"Labels/Whales_Full/{name}.jpg")
#     except:print(f"No file found: {name}")

In [46]:
dataset=keras.preprocessing.image_dataset_from_directory('Labels', batch_size=50, image_size=(150,150))

Found 51033 files belonging to 2 classes.


In [23]:
for data, label in dataset:
    print(data.shape)
    print(data.dtype)
    print(label.shape)
    print(label.dtype)

(20, 150, 150, 3)
<dtype: 'float32'>
(20,)
<dtype: 'int32'>
(20, 150, 150, 3)
<dtype: 'float32'>
(20,)
<dtype: 'int32'>
(20, 150, 150, 3)
<dtype: 'float32'>
(20,)
<dtype: 'int32'>
(20, 150, 150, 3)
<dtype: 'float32'>
(20,)
<dtype: 'int32'>
(20, 150, 150, 3)
<dtype: 'float32'>
(20,)
<dtype: 'int32'>
(20, 150, 150, 3)
<dtype: 'float32'>
(20,)
<dtype: 'int32'>
(20, 150, 150, 3)
<dtype: 'float32'>
(20,)
<dtype: 'int32'>
(12, 150, 150, 3)
<dtype: 'float32'>
(12,)
<dtype: 'int32'>


In [47]:
from tensorflow.keras import layers
from tensorflow.keras.layers import LayerNormalization

model=keras.models.Sequential()
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Flatten(input_shape=[150,150,3]))
model.add(keras.layers.Dense(150, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))

In [48]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [49]:
model.fit(dataset, epochs=5)

Epoch 1/5
1021/1021 [==============================] - 1536s 2s/step - loss: 1.2256 - accuracy: 0.7371
Epoch 2/5
1021/1021 [==============================] - 2061s 2s/step - loss: 0.5612 - accuracy: 0.7842
Epoch 3/5
1021/1021 [==============================] - 1543s 2s/step - loss: 0.5154 - accuracy: 0.7833
Epoch 4/5
1021/1021 [==============================] - 1601s 2s/step - loss: 0.4682 - accuracy: 0.8020
Epoch 5/5
1021/1021 [==============================] - 1569s 2s/step - loss: 0.4443 - accuracy: 0.8084


## Separate Dolphins into species of Dolphin

In [14]:
Dolphins_Species=pd.DataFrame(Labels_DF.loc[Labels_DF['sub_cat']==1])

In [38]:
Dolphins_Species['species_category'].value_counts()

2     10781
7      3139
25     1700
26      490
5       347
27      229
18      145
4        90
21       60
10       14
Name: species_category, dtype: int64

#### Consider combining categories with low counts for better training when trying to identify individuals

In [37]:
len(Labels_DF['individual_id'].unique())

15587

####  15,587 unique individuals in entire data set (dolphins and whales) 51,000 images.  Cannot make individual id files.
####  Species category 10 has 13 unique individuals out of 14 instances.